<a href="https://colab.research.google.com/github/sarunsmenon/llm/blob/main/chandler_park_primary_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [1]:
!pip install -q python-dotenv openai langchain-openai cohere langchain langchain_community pypdf faiss-gpu wikipedia-api faiss-cpu wikipedia langchainhub unstructured playwright uuid7

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.7/336.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Load Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import textwrap

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.messages import BaseMessage, AIMessage

In [4]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
from google.colab import userdata
import pickle

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader, WikipediaLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool

from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

from langchain import hub
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

from uuid_extensions import uuid7str
from langchain_core.pydantic_v1 import BaseModel, Field


# Load Variables

In [6]:
os.environ['OPENAI_API_KEY'] = userdata.get('open_ai_key')
prompt_template = "hwchase17/openai-functions-agent"

os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_API_KEY']=userdata.get('langsmith_api_key')

session_id = uuid7str()

In [7]:
llm_model = 'gpt-3.5-turbo-1106'
llm = ChatOpenAI(model=llm_model, temperature=0)

In [8]:
# Start crawling from the initial URL
start_url = 'https://chandlerparkps.vic.edu.au/'
ignore_lst = []
include_lst = ['support' ,'telstra']
max_pg_lmt = 5000
db_name = "faiss_cpps_support_db"
fldr = '/content/drive/MyDrive/Colab Notebooks/Langchain/cpps_support/'

In [9]:
hist_store= {}

In [10]:
if not os.path.exists(fldr):
    # If the folder does not exist, create it
    os.makedirs(fldr)
    print(f'Folder created at: {fldr}')
else:
    print(f'Folder already exists at: {fldr}')

Folder already exists at: /content/drive/MyDrive/Colab Notebooks/Langchain/cpps_support/


# Load Functions

In [11]:
def load_history():
  with open(fldr+"history.pkl", "rb") as f:
    hist_store = pickle.load(f)

In [12]:
def store_history():
  with open(fldr+"history.pkl", "wb") as f:
    pickle.dump(hist_store, f)

In [13]:
# Function to get all links from a page
def get_all_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = [a.get('href') for a in soup.find_all('a', href=True)]
    full_links = [urljoin(url, link) for link in links]
    return full_links

In [14]:
print(max_pg_lmt)

5000


In [15]:
# Function to crawl the website
def crawl_website(start_url, max_pages=max_pg_lmt):
    itr = 0

    visited = set()
    to_visit = [start_url]

    while to_visit and len(visited) < max_pages:
      url = to_visit.pop(0)

      if (
          (url not in visited) and
          ("chandlerparkps.vic.edu.au" in url)
        ):
        visited.add(url)
        try:
          links = get_all_links(url)
          to_visit.extend(links)
        except:
          continue

        itr += 1
        if itr % 10 == 0:
          print(f"Visited {len(visited)}: {url}")

    return visited

In [28]:
text_splitter = CharacterTextSplitter(
              chunk_size=1000,
              chunk_overlap=10,
              separator= "\n\n",
              length_function=len,
              is_separator_regex=False
            )

In [16]:
def extract_process_url(url):
  loader = UnstructuredURLLoader(urls=[url])
  data = loader.load()

  docs = text_splitter.split_documents(data)
  return docs


In [27]:
def extract_process_pdf(url):
  loader = PyPDFLoader(file_path)
  pages = loader.load_and_split(text_splitter=text_splitter)

  return pages


In [17]:
def store_doc_into_db(docs, faiss_db):
  if faiss_db is None:
    faiss_db = FAISS.from_documents(docs, OpenAIEmbeddings())
  else:
    faiss_db.add_documents(docs)

  return faiss_db


In [18]:
def generate_chat_response(message, local_session_id):
  result = agent_with_chat_history.invoke({"input": message}, config={"configurable": {"session_id": local_session_id}})
  return result['output']

In [19]:
def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in hist_store:
        hist_store[session_id] = InMemoryHistory()
    return hist_store[session_id]


In [20]:
def get_user_and_retrieve_history():
  # user_name = input("Enter your username : ")
  history = get_by_session_id(user_name)
  return history

def get_user():
  user_name = input("Enter your username : ")
  return user_name

In [21]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# Load History

In [23]:
if os.path.exists(fldr+"history.pkl"):
  hist_store = load_history()
  print(f'Folder already exists at: {fldr}')

# Create URL Lists

In [31]:
visited_urls = crawl_website(start_url)
print(f"Total visited URLs: {len(visited_urls)}")

Visited 10: https://chandlerparkps.vic.edu.au/curriculum/
Visited 20: https://chandlerparkps.vic.edu.au/wellbeing/#wellbeing
Visited 30: https://chandlerparkps.vic.edu.au/parents/


Visited 40: https://chandlerparkps.vic.edu.au/wp-content/uploads/2024/04/IMG_7861.jpg


Visited 50: https://chandlerparkps.vic.edu.au/wp-content/uploads/2022/05/Enrolment-Information.pdf


Visited 60: https://chandlerparkps.vic.edu.au/wp-content/uploads/2023/11/BOOK-PACK-ORDER-FORM-YEAR-2-2024.pdf


Visited 70: https://chandlerparkps.vic.edu.au/wp-content/uploads/2023/11/BOOK-PACK-ORDER-FORM-YEAR-6-2024-1.pdf


...

Visited 90: https://chandlerparkps.vic.edu.au/wp-content/uploads/2021/08/Hire-of-Facilities.pdf


Visited 100: https://chandlerparkps.vic.edu.au/wp-content/uploads/2021/08/School-Commitment-Pledge.pdf


Visited 110: https://chandlerparkps.vic.edu.au/2019/01/22/hello-world/


/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


Total visited URLs: 115


In [32]:
with open(fldr+"url_list.pkl", "wb") as f:
    pickle.dump(visited_urls, f)

In [24]:
with open(fldr+"url_list.pkl", "rb") as f:
    visited_urls = list(pickle.load(f))

error_url_lst = []

In [25]:
print(f"Total visited URLs: {len(visited_urls)}")

Total visited URLs: 115


# Create Database

In [31]:
if os.path.exists(fldr+db_name):
  faiss_data_db = FAISS.load_local(fldr+db_name,embeddings=OpenAIEmbeddings(), allow_dangerous_deserialization=True)
  print("DB already exists")
else:
  faiss_data_db = None
  print("create a new database because none exists")


create a new database because none exists


In [32]:
for url in visited_urls[0:5000]:
  print(url)
  try:
    if "pdf" in url:
      docs = extract_process_pdf(url)
    else:
      docs = extract_process_url(url)

    faiss_data_db = store_doc_into_db(docs, faiss_data_db)
  except:
    error_url_lst.append(url)


https://chandlerparkps.vic.edu.au/wp-content/uploads/2021/08/Inclusion-and-Diversity-Policy.pdf
https://chandlerparkps.vic.edu.au/about-us/#zone


https://chandlerparkps.vic.edu.au/about-us/#council


https://chandlerparkps.vic.edu.au/wp-content/uploads/2023/02/Privacy_Collection_Notice.pdf
https://chandlerparkps.vic.edu.au/wp-content/uploads/2021/08/Child-Safety-Responding-and-Reporting-Obligations-Policy-and-Procedures.pdf
https://chandlerparkps.vic.edu.au/wp-content/uploads/2024/02/gwfe-information-for-parents.pdf
https://chandlerparkps.vic.edu.au/wp-content/uploads/2022/10/2022-Child-Safe-Code-of-Conduct.docx.pdf
https://chandlerparkps.vic.edu.au/wp-content/uploads/2021/08/Parent-Disputes-Policy.pdf
https://chandlerparkps.vic.edu.au/enrolment/#parenttransition
https://chandlerparkps.vic.edu.au/wp-content/uploads/2023/11/02-2024-Year-2-Parent-Payments.pdf
https://chandlerparkps.vic.edu.au/2019/01/
https://chandlerparkps.vic.edu.au/wp-content/uploads/2022/11/2022-Uniform-Policy.pdf
https://chandlerparkps.vic.edu.au/curriculum/#inquiry
https://chandlerparkps.vic.edu.au/wp-content/uploads/2023/11/01-2024-Year-1-Parent-Payments-2.pdf
https://chandlerparkps.vic.edu.au/wp-content/uploa

ERROR:langchain_community.document_loaders.url:Error fetching or processing https://chandlerparkps.vic.edu.au/wp-content/uploads/2024/04/IMG_7814.jpg, exception: partition_image() is not available because one or more dependencies are not installed. Use: pip install "unstructured[image]" (including quotes) to install the required dependencies


https://chandlerparkps.vic.edu.au/wp-content/uploads/2022/10/2022-Complaints-Policy.pdf
https://chandlerparkps.vic.edu.au/enrolment/#enrolment
https://chandlerparkps.vic.edu.au/wp-content/uploads/2021/08/Working-with-Children-Check-Policy.pdf
https://chandlerparkps.vic.edu.au/wp-content/uploads/2023/06/Chandler-Park-Primary-School-5533-2021-2024-School-Strategic-Plan-A3.pdf
https://chandlerparkps.vic.edu.au/about-us/#profile


...

https://chandlerparkps.vic.edu.au/wp-content/uploads/2021/10/Student-Engagement-and-Wellbeing-Policy-1.pdf
https://chandlerparkps.vic.edu.au/wp-content/uploads/2021/08/Anaphylaxis-Policy.pdf
https://chandlerparkps.vic.edu.au/wp-content/uploads/2023/11/CHANPPS-ORDER-FORM-PREP-2024.docx


ERROR:langchain_community.document_loaders.url:Error fetching or processing https://chandlerparkps.vic.edu.au/wp-content/uploads/2023/11/CHANPPS-ORDER-FORM-PREP-2024.docx, exception: partition_docx() is not available because one or more dependencies are not installed. Use: pip install "unstructured[docx]" (including quotes) to install the required dependencies


https://chandlerparkps.vic.edu.au/wp-content/uploads/2022/10/Enrolling-in-Primary-School-Letter-updated-28-March-2022.pdf
https://chandlerparkps.vic.edu.au/wp-content/uploads/2022/10/2022-Yard-Duty-and-Supervision-Policy.pdf
https://chandlerparkps.vic.edu.au/wp-content/uploads/2021/10/Curriculum-Framework-Policy.pdf
https://chandlerparkps.vic.edu.au/2019/01/22/hello-world/
https://chandlerparkps.vic.edu.au/parents/#council
https://chandlerparkps.vic.edu.au/wp-content/uploads/2024/04/IMG_7807.jpg


ERROR:langchain_community.document_loaders.url:Error fetching or processing https://chandlerparkps.vic.edu.au/wp-content/uploads/2024/04/IMG_7807.jpg, exception: partition_image() is not available because one or more dependencies are not installed. Use: pip install "unstructured[image]" (including quotes) to install the required dependencies


https://chandlerparkps.vic.edu.au/wp-content/uploads/2023/05/Student-Application-Form-2024-PDF-Version.pdf
https://chandlerparkps.vic.edu.au/wp-content/uploads/2023/11/Chandler-park-Theircare-flyer-English-1.pdf


# Write Everything

In [33]:
print(faiss_data_db)

In [34]:
FAISS.save_local(faiss_data_db, fldr+db_name)

In [35]:
with open(fldr+"error_urls.pkl", "wb") as f:
  pickle.dump(error_url_lst, f)

# Create Retriever

In [36]:
if os.path.exists(fldr+db_name):
  faiss_data_db = FAISS.load_local(fldr+db_name,embeddings=OpenAIEmbeddings(), allow_dangerous_deserialization=True)
else:
  faiss_data_db = None

In [37]:
primary_retriever = faiss_data_db.as_retriever( search_type="similarity_score_threshold",
                            search_kwargs={"score_threshold": 0.5,"k": 1}
)

In [45]:
primary_retriever.invoke("who is school principal")

[Document(metadata={'source': 'https://chandlerparkps.vic.edu.au/about-us/#values'}, page_content='Jo Hillman\n\nPrincipal\n\nSchool Profile\n\nAt Chandler Park we encourage and foster positive attitudes towards learning. We have high expectations of success, a collaborative focus on improving student learning outcomes and an inclusive school environment in which the development of every child is our priority.\n\nWe have an expectation that students will have:\n\nA sense of their own learning strengths- using learning styles, effective learning behaviours, concentration and resilience\n\nOrganisational skills and strategies to work independently\n\nAn emerging repertoire of learning strategies to use in a variety of situations\n\nSelf-confidence to persist in the acquisition of literacy and numeracy skills\n\nThe skills required to research, organise and present data\n\nThe skills required to be a reflective learner.')]

# Agents

In [39]:
agent_tool = create_retriever_tool(
    primary_retriever,
    "cpps_support_search",
    "Search for information on Chandler park primary school also known as CPPS"
)

In [40]:
tools = [agent_tool ]

In [41]:
# Get the prompt to use - you can modify this!
prompt = hub.pull(prompt_template)
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [42]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

# Chat History

In [43]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_user_and_retrieve_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Execution

In [44]:

user_name = get_user()
print(f"welcome :: {user_name}")


try:
  input_qn  = "Hi Bot, I need help with Chandler Park Primary School"
  while input_qn != "exit":

    print("start chat")
    response = generate_chat_response(input_qn, user_name)
    print(textwrap.fill(response, 80))
    print("")
    input_qn = input("Enter a message (to finish use exit): ")
except:
  print(traceback.format_exc())
finally:
  store_history()

Enter your username : samarth
welcome :: samarth
start chat
Chandler Park Primary School is accredited under the Department of Education’s
CRICOS registration. For more information, you can visit www.study.vic.gov.au.
If you need further assistance, feel free to ask!

Enter a message (to finish use exit): what is uniform
start chat
The Chandler Park Primary School uniform is available for purchase from the PSW
Retail Outlet at the following address:  Unit 1, 9-11 South Link Dandenong South
VIC 3175  You can also find the uniform price list and policy at the PSW Uniform
Shop. Additionally, reversible house-colored hats are available for purchase
from the school office in sizes small, medium, large, and extra-large.  If you
have any further questions or need more information, feel free to ask!

Enter a message (to finish use exit): what is numeracy curriculum for year 3
start chat
The numeracy curriculum for Year 3 at Chandler Park Primary School is organized
around the three content str

# Experiment Section

In [64]:
for message in hist_store['samarth'].messages:
  result = []
  temp_pair = []

  for message in hist_store['samarth'].messages:
      if message.type == 'human':
          if temp_pair:  # Check if there is already a user message in the temp_pair
              result.append(temp_pair)  # Add the incomplete pair (just a user message)
              temp_pair = []  # Reset the temp_pair
          temp_pair.append(message.content)  # Add user message to temp_pair
      elif message.type == 'ai':
          if len(temp_pair) == 1:  # Ensure there is a user message before adding AI message
              temp_pair.append(message.content)  # Add AI message to complete the pair
              result.append(temp_pair)  # Add the complete pair to result
              temp_pair = []  # Reset the temp_pair

  # Handle case where the last message is a user message without a paired AI message
  if temp_pair:
      result.append(temp_pair)

print(result)

[['Hi Bot, I need help with Chandler Park Primary School', 'Chandler Park Primary School is accredited under the Department of Education’s CRICOS registration. For more information, you can visit www.study.vic.gov.au. If you need further assistance, feel free to ask!'], ['what is uniform', 'The Chandler Park Primary School uniform is available for purchase from the PSW Retail Outlet at the following address:\n\nUnit 1, 9-11 South Link\nDandenong South VIC 3175\n\nYou can also find the uniform price list and policy at the PSW Uniform Shop. Additionally, reversible house-colored hats are available for purchase from the school office in sizes small, medium, large, and extra-large.\n\nIf you have any further questions or need more information, feel free to ask!'], ['what is numeracy curriculum for year 3', 'The numeracy curriculum for Year 3 at Chandler Park Primary School is organized around the three content strands and four proficiency strands of the Victorian Curriculum. The content st

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr

os.environ["OPENAI_API_KEY"] = "sk-..."  # Replace with your key

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-0613')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm(history_langchain_format)
    return gpt_response.content

gr.ChatInterface(predict).launch()

In [53]:
hist_store['samarth'].messages

[HumanMessage(content='Hi Bot, I need help with Chandler Park Primary School'),
 AIMessage(content='Chandler Park Primary School is accredited under the Department of Education’s CRICOS registration. For more information, you can visit www.study.vic.gov.au. If you need further assistance, feel free to ask!'),
 HumanMessage(content='what is uniform'),
 AIMessage(content='The Chandler Park Primary School uniform is available for purchase from the PSW Retail Outlet at the following address:\n\nUnit 1, 9-11 South Link\nDandenong South VIC 3175\n\nYou can also find the uniform price list and policy at the PSW Uniform Shop. Additionally, reversible house-colored hats are available for purchase from the school office in sizes small, medium, large, and extra-large.\n\nIf you have any further questions or need more information, feel free to ask!'),
 HumanMessage(content='what is numeracy curriculum for year 3'),
 AIMessage(content='The numeracy curriculum for Year 3 at Chandler Park Primary Sch